Modelul este prea mare, nu s-a ajuns pana la finalul rularii programului

In [ ]:

!pip install -q transformers accelerate sentencepiece torch


MODEL_NAME = "mistralai/Mistral-7B-Instruct-v0.3"

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)


SYSTEM_MSG = "Ești un asistent care redactează cereri clare, politicoase și bine structurate în limba română."

TEMPLATE = """
Creează o cerere în limba română pe baza elementelor de mai jos:
- TipCerere: {tip}
- Destinatar: {destinatar}
- Expeditor: {expeditor}
- Scop: {scop}
- Argumente: {argumente}
- Termen dorit: {termen}
- Ton: {ton}
- Lungime: {lungime} cuvinte
"""

import random

types = ["cerere de concediu", "cerere de finanțare", "cerere de suport tehnic"]
destinatari = ["departament HR", "director financiar", "echipa IT"]
expeditori = ["angajat", "manager de proiect", "student"]
scopuri = ["obținerea concediului medical", "finanțarea proiectului X", "rezolvarea unei erori software"]
argumente = ["atașez documentele necesare", "am completat formularul cerut", "includ detalii tehnice suplimentare"]
termeni = ["urgent", "până la 10 mai", "în cel mult o săptămână"]
tonuri = ["formal", "semi-formal"]
lungimi = [150, 250]

def sample_prompt():
    return TEMPLATE.format(
        tip=random.choice(types),
        destinatar=random.choice(destinatari),
        expeditor=random.choice(expeditori),
        scop=random.choice(scopuri),
        argumente=random.choice(argumente),
        termen=random.choice(termeni),
        ton=random.choice(tonuri),
        lungime=random.choice(lungimi)
    )


def call_model(system, prompt, temperature=0.4, max_new_tokens=400):
    full_prompt = f"System: {system}\nUser: {prompt}\nAssistant:"
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


for i in range(3):
    p = sample_prompt()
    print(f"\n=== Prompt {i+1} ===\n{p}")
    out = call_model(SYSTEM_MSG, p)
    print(f"\n--- Cerere generată ---\n{out}\n")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/587k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/4.55G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]


=== Prompt 1 ===

Creează o cerere în limba română pe baza elementelor de mai jos:
- TipCerere: cerere de suport tehnic
- Destinatar: echipa IT
- Expeditor: student
- Scop: rezolvarea unei erori software
- Argumente: atașez documentele necesare
- Termen dorit: urgent
- Ton: semi-formal
- Lungime: 150 cuvinte



KeyboardInterrupt: 